In [87]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [88]:
if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
login = 114781990
password = 'Mypassword$1234'
server = 'Exness-MT5Trial6'

mt5.login(login, password, server)

rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 5000)

nf = pd.DataFrame(rate)
#         print('getting data')

nf['date']=pd.to_datetime(nf['time'], unit='s')

nf.set_index(np.arange(len(nf)), inplace = True)
nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


# nf.to_excel('EURUSD.xlsx')

df = nf.iloc[:500]  # First half
df2 = nf.iloc[500:]  # Second half

In [89]:
def get_kc( high, low, close, kc_lookback, multiplier, atr_lookback):
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift()))
    tr3 = pd.DataFrame(abs(low - close.shift()))

    # Calculate True Range as the element-wise maximum of the three components
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/atr_lookback).mean()

    kc_middle = close.ewm(span=kc_lookback).mean()
    kc_upper = kc_middle + multiplier * atr
    kc_lower = kc_middle - multiplier * atr

    return kc_middle.iloc[-1], kc_upper.iloc[-1], kc_lower.iloc[-1]

def candle(df):
    '''Return candle colour'''

    if df['close'] - df['open'] > 0:
        return 'G'

    else:
        return 'R'
    
    
def calculate_indicators():

#         print('calc_indicator')
    global df
    length = len(df) - 1

    df.loc[length, 'EMA_8'] = talib.EMA(df['close'], timeperiod=8).iloc[-1]
    df.loc[length, 'EMA_50'] = talib.EMA(df['close'], timeperiod=50).iloc[-1]
    df.loc[length, 'EMA_200'] = talib.EMA(df['close'], timeperiod=200).iloc[-1]
    df.loc[length, 'RSI'] = talib.RSI(df['close'], timeperiod=21).iloc[-1]

    df.loc[length, 'candle'] = candle(df.iloc[-1])

    df.loc[length, 'EMA_20'], df.loc[length, 'upper_keltner'], df.loc[length, 'lower_keltner'] = get_kc(df['high'], df['low'], df['close'], kc_lookback=20, multiplier=2.25, atr_lookback=10)



def check_body( df):
    #     Calculate 0.7 % of entry price
#     fOR 1H FOREX
        one_percent = df['open']/100 
        percent = one_percent * 0.06
        hammer_per = one_percent * 0.06


    #     Calculate candle size close-open
        if df['candle'] == 'G':
            candle_size = df['close'] - df['open']
            upper_wick = df['high'] - df['close']
            lower_wick = df['open'] - df['low']

        else:
            candle_size = abs(df['close'] - df['open'])
            upper_wick = df['high'] - df['open']
            lower_wick = df['close'] - df['low']

        upper_wick_per = (upper_wick/one_percent) 
        lower_wick_per = (lower_wick/one_percent) 



    #     check for entry candle size and Max candle size 0.12 %
        if ((candle_size >= percent) and candle_size <= (0.12 * one_percent) and
         (candle_size > upper_wick) and (candle_size > lower_wick) and
        (upper_wick_per < 0.06) and (lower_wick_per < 0.06)):

            if df['candle'] == 'G' and df['high'] < df['upper_keltner'] :

                return 1

            elif df['candle'] == 'R' and df['low'] > df['lower_keltner'] :

                return 1

            else:

                return 0

    #     condition for hammmer
    #     lower wick is atleast 50 % of total size 
    #     Green Hammer 
    # need to add max wick of opposite dir
        elif ( (df['candle'] == 'G') and (df['open'] - df['low']) >= ((df['high'] - df['low']) * 0.5) and
              (candle_size >= hammer_per) and (candle_size > upper_wick)and
              candle_size <= (0.12 * one_percent) and ((upper_wick_per < 0.06))  
             and df['high'] < df['upper_keltner']):

            return 1


#     For Red Hammer
# need to add max wick of opposite dir
        elif ( (df['candle'] == 'R') and (df['high'] - df['open']) >= (abs(df['high'] - df['low']) * 0.5) and 
              (candle_size >= hammer_per) and (candle_size > lower_wick)and candle_size <= (0.12 * one_percent) and 
              ((lower_wick_per < 0.06)) and df['low'] > df['lower_keltner'] ):

            return 1


        else:
            return 0


In [90]:
def check_inside_kt( x):

    if  ( x['high'] < x['upper_keltner'] and x['low'] > x['lower_keltner']):
        return 1

    else:
        return 0

In [91]:
def check_prev_candle(entry_candle):
    
    global df

    entry_index = df[df['date'] == entry_candle['date']].index[0]
    entry_candle_size = abs(entry_candle['open'] - entry_candle['close'])

    if entry_candle['candle'] == 'G':
        for i in range(entry_index - 1, max(-1, entry_index - 11), -1):
            current_candle = df.iloc[i]
            current_candle_size = abs(current_candle['open'] - current_candle['close'])

            if (current_candle['candle'] == 'R' and
                current_candle['open'] <= entry_candle['close'] and
                current_candle_size >= 0.5 * entry_candle_size and 
                current_candle_size <= 1.2 * entry_candle_size):
                
                for j in range(i + 1, entry_index):
                    intermediate_candle = df.iloc[j]
                    
                    if (abs(intermediate_candle['open'] - intermediate_candle['close']) <= 0.6 *
                        current_candle_size and j < entry_index - 1):
                        continue
                    elif j == entry_index - 1:
                        return True
                    else:
                        return False
                break

    elif entry_candle['candle'] == 'R':
#         print('Testing R candle')
        for i in range(entry_index - 1, max(-1, entry_index - 11), -1):
            current_candle = df.iloc[i]
            current_candle_size = abs(current_candle['open'] - current_candle['close'])

            if (current_candle['candle'] == 'G' and
                current_candle['open'] >= entry_candle['close'] and
                current_candle_size >= 0.5 * entry_candle_size and 
                current_candle_size <= 1.2 * entry_candle_size):
#                 print('R in IF')
                for j in range(i + 1, entry_index):
                    intermediate_candle = df.iloc[j]
                    
                    if (abs(intermediate_candle['open'] - intermediate_candle['close']) <= 0.6 *
                        current_candle_size and j < entry_index - 1):
                        continue
                    
                    elif j == entry_index - 1:
                        return True
                    
                    else:
                        return False
                    
                break

    return False


In [92]:
# Condition Check

checker = pd.DataFrame()

def check_conditions():

    global df, checker
    length = len(df) - 1

#         Checking all entry conditions

    df.loc[length, 'candle'] = candle(df.iloc[-1])
    
    df.loc[length, 'pos'] = check_inside_kt(df.iloc[-1])

    df.loc[length, 'candle_size'] = check_body(df.iloc[-1])
    
#     print('Candle: ',df.loc[length, 'candle'] )
#     print('KT: ', df.loc[length, 'pos'])
#     print('Size: ', df.loc[length, 'candle_size'])

    df.loc[length, 'prev_candle'] = check_prev_candle(df.iloc[-1])
    
#     print('Prev candle: ', df.loc[length, 'prev_candle'])
    
    if (df.loc[length, 'pos'] == 1 and 
        df.loc[length, 'candle_size'] == 1 and df.loc[length, 'prev_candle'] == True):
    
#         print('Complete')
        print('DF: ', df.iloc[-1])


In [93]:
#  Caller
past_df = pd.DataFrame()
def Caller():
    
    global df, df2, past_df
    # Initialize an empty DataFrame to store the past data that gets dropped
    

#         Loop for taking the live data
    for index, row in df2.iterrows():
        # Add live data to the df DataFrame
        df = df.append(row, ignore_index=True)
#             self.df = pd.concat([self.df, row], ignore_index=True)

        # Delete the first row if the DataFrame has more than 202 rows
        if len(df) > 500:
            # Save the dropped row in past_df
            past_df = past_df.append(df.iloc[0], ignore_index=True)
            df = df.iloc[1:].reset_index(drop=True)
            
            
        calculate_indicators()

        check_conditions()

    result_df = pd.concat([past_df, df], ignore_index=True)

    result_df.to_excel('combined.xlsx')

In [94]:
Caller()



C:\Users\hp\AppData\Local\Temp\ipykernel_7052\4103600624.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_7052\4103600624.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_df = past_df.append(df.iloc[0], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_7052\4103600624.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_7052\4103600624.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_df = past_df.append(df.iloc[0], ignore_index=True)


DF:  open                         1.12568
high                         1.12574
low                          1.12464
close                        1.12485
date             2023-07-18 11:00:00
EMA_8                       1.125018
EMA_50                       1.12477
EMA_200                     1.122862
RSI                        49.979112
candle                             R
EMA_20                      1.124973
upper_keltner               1.126955
lower_keltner               1.122991
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.12292
high                         1.12292
low                          1.12198
close                        1.12198
date             2023-07-19 10:00:00
EMA_8                        1.12282
EMA_50                       1.12265
EMA_200                     1.122746
RSI                        46.541717
candle                             R
EMA

C:\Users\hp\AppData\Local\Temp\ipykernel_7052\4103600624.py:18: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  past_df = past_df.append(df.iloc[0], ignore_index=True)


DF:  open                         1.11219
high                         1.11319
low                          1.11213
close                        1.11315
date             2023-07-24 05:45:00
EMA_8                        1.11243
EMA_50                      1.112529
EMA_200                     1.114507
RSI                         55.73213
candle                             G
EMA_20                      1.112451
upper_keltner               1.113485
lower_keltner               1.111416
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.10813
high                         1.10896
low                          1.10802
close                        1.10892
date             2023-07-24 10:30:00
EMA_8                       1.108522
EMA_50                      1.110824
EMA_200                     1.113666
RSI                        42.261037
candle                             G
EMA

DF:  open                         1.10095
high                         1.10097
low                          1.10008
close                        1.10024
date             2023-08-07 01:00:00
EMA_8                       1.100582
EMA_50                      1.100277
EMA_200                     1.097627
RSI                        47.661762
candle                             R
EMA_20                      1.100696
upper_keltner                1.10173
lower_keltner               1.099661
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                          1.0947
high                         1.09486
low                          1.09376
close                        1.09396
date             2023-08-08 13:30:00
EMA_8                       1.094561
EMA_50                      1.096983
EMA_200                      1.09817
RSI                         32.01643
candle                             R
EMA

DF:  open                          1.0892
high                         1.08932
low                          1.08787
close                        1.08803
date             2023-08-16 18:45:00
EMA_8                       1.088839
EMA_50                      1.090321
EMA_200                     1.091655
RSI                        38.288683
candle                             R
EMA_20                      1.089451
upper_keltner               1.091598
lower_keltner               1.087305
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.09092
high                         1.09092
low                          1.08972
close                         1.0899
date             2023-08-17 13:45:00
EMA_8                       1.090298
EMA_50                       1.08856
EMA_200                     1.089701
RSI                        58.263744
candle                             R
EMA

DF:  open                         1.09275
high                         1.09276
low                          1.09188
close                          1.092
date             2023-08-30 17:45:00
EMA_8                       1.092563
EMA_50                      1.090513
EMA_200                     1.086382
RSI                        56.512843
candle                             R
EMA_20                      1.092199
upper_keltner               1.093866
lower_keltner               1.090531
pos                              1.0
candle_size                      1.0
prev_candle                     True
Name: 499, dtype: object
DF:  open                         1.08678
high                           1.088
low                          1.08649
close                        1.08789
date             2023-08-31 12:00:00
EMA_8                       1.087168
EMA_50                      1.089473
EMA_200                     1.088533
RSI                        41.391905
candle                             G
EMA